# Run Moatless Tools

First, index the codebase in a vector store.

Set `repo_dir` to the path of the repository you want to index.

In [1]:
from moatless.index import CodeIndex, IndexSettings
from moatless.benchmark.swebench import setup_swebench_repo, get_repo_dir_name
from moatless import FileRepository, Settings, Workspace

import os

# An OPENAI_API_KEY is required to use the OpenAI Models
Settings.agent_model = "gpt-4o-2024-05-13"
index_settings = IndexSettings(
    embed_model="text-embedding-3-small"
)

repo_dir = "/tmp/moatless"
file_repo = FileRepository(repo_path=repo_dir)

code_index = CodeIndex(file_repo=file_repo, settings=index_settings)
nodes, tokens = code_index.run_ingestion()

print(f"Indexed {nodes} nodes and {tokens} tokens")

workspace = Workspace(file_repo=file_repo, code_index=code_index)

Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/177 [00:00<?, ?it/s]

Indexed 177 nodes and 58057 tokens


Then use the `SearchLoop` to find the relevant code.

In [2]:
from moatless.loop import SearchLoop

instructions = "Remove the token limit check from the completion funktion"

search_loop = SearchLoop(workspace, instructions=instructions)
search_response = search_loop.execute()

for file in search_response.files:
    print(f"File: {file.file_path}")
    print(f"Spans:")
    for span_id in file.span_ids:
        print(f" - {span_id}")

File: moatless/llm/completion.py
Spans:
 - completion


Execute the `CodeLoop` to apply the changes.

In [3]:
from moatless.loop import  CodeLoop

coder = CodeLoop(workspace, instructions=instructions, files=search_response.files)
code_response = coder.execute()

print(f"Response: {code_response.message}")

Response: The token limit check has been successfully removed from the completion function as requested.


Run a `$ git diff` to see the changes.

In [4]:
import subprocess

output = subprocess.run(
      ["git", "diff"],
      capture_output=True,
      text=True,
      cwd=repo_dir,
)

print(output.stdout)

diff --git a/moatless/llm/completion.py b/moatless/llm/completion.py
index 2c95e47..a926948 100644
--- a/moatless/llm/completion.py
+++ b/moatless/llm/completion.py
@@ -48,10 +48,6 @@ def completion(
 
     metadata["trace_name"] = trace_name
 
-    tokens = token_counter(messages=messages[-1:])
-    if tokens > Settings.max_message_tokens:
-        raise ValueError(f"Too many tokens in the new message: {tokens}")
-
     response = litellm.completion(
         model=model,
         max_tokens=max_tokens,

